COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Owen McLachalan
#### Date: Jan 29

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [74]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [75]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

KeyError: 'element_count'

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [ ]:
# Ex.1 your code here
class Asteroid_1():

    def __init__(self, close_approach_date, miss_distance, orbiting_body, relative_velocity):


      self.close_approach_date = close_approach_date
      self.miss_distance = miss_distance
      self.orbiting_body = orbiting_body
      self.relative_velocity = relative_velocity

    def __str__(self):
      main_str = f"""On {self.close_approach_date}, a object orbiting {self.orbiting_body} at {self.relative_velocity} m/s came {self.miss_distance}km from Earth."""
      return main_str
    def __repr__(self):
      mainer_str = f"""On {self.close_approach_date}, a object orbiting {self.orbiting_body} at {self.relative_velocity} m/s came {self.miss_distance}km from Earth."""
      return mainer_str

asteroid_x = {'close_approach_date': '1949-07-04',
                 'close_approach_date_full': '1949-Jul-04 22:19',
                 'epoch_date_close_approach': -646710060000,
                 'miss_distance': {'astronomical': '0.419726307',
                                   'kilometers': '62790161.51016609',
                                   'lunar': '163.273533423',
                                   'miles': '39015997.166588442'},
                 'orbiting_body': 'Earth',
                 'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                       'kilometers_per_second': '20.4193538659',
                                       'miles_per_hour': '45676.0526626122'}} # first entery dict from site


asteroid_test_instance = Asteroid_1(asteroid_x['close_approach_date'], asteroid_x['miss_distance']['kilometers'], asteroid_x['orbiting_body'], asteroid_x['relative_velocity']['kilometers_per_hour'])

print(str(asteroid_test_instance))
print(repr(asteroid_test_instance))





## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [ ]:
# Ex. 2 your code here


class Asteroid():
    def __init__(self, close_approach_date, miss_distance, orbiting_body, relative_velocity, id):
        self.close_approach_date = close_approach_date
        self.miss_distance = miss_distance
        self.orbiting_body = orbiting_body
        self.relative_velocity = relative_velocity

    @classmethod
    def from_NEO(cls, neo_id):
        API_KEY = 'DEMO_KEY'

        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
        print(url)
        response = requests.get(url)
        neo_data = json.loads(response.text)

        # Extract relevant information from the API response
        close_approach_date = neo_data['close_approach_data'][0]['close_approach_date']
        miss_distance = neo_data['close_approach_data'][0]['miss_distance']['kilometers']
        orbiting_body = neo_data['close_approach_data'][0]['orbiting_body']
        relative_velocity = neo_data['close_approach_data'][0]['relative_velocity']['kilometers_per_second']



        return cls(close_approach_date, miss_distance, orbiting_body, relative_velocity, id)



neo_id_to_fetch = "3542518"  # wrong IDs cause failure!!!!!!
asteroid = Asteroid.from_NEO(neo_id_to_fetch)

print(asteroid.close_approach_date)
print(asteroid.miss_distance)
print(asteroid.orbiting_body)
print(asteroid.relative_velocity)

print("------------------------------------")

neo_id_to_fetch = "3542517"  # wrong IDs cause failure!!!!!!
asteroid = Asteroid.from_NEO(neo_id_to_fetch)


print(asteroid.close_approach_date)
print(asteroid.miss_distance)
print(asteroid.orbiting_body)
print(asteroid.relative_velocity)
#print(asteroid.id)





## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [ ]:
# Ex. 3 your code here


class CloseApproach():
    def __init__(self, orbiting_body, approach_date, miss_distance, relative_velocity, neo_id):
        self.neo_id = neo_id
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity



    @classmethod
    def close_approach_data(cls, close_approach_data, neo_id):
        orbiting_body = close_approach_data['orbiting_body']
        approach_date = close_approach_data['close_approach_date']#if error do not change
        miss_distance = float(close_approach_data['miss_distance']['kilometers'])
        relative_velocity = float(close_approach_data['relative_velocity']['kilometers_per_second'])
        neo_id = neo_id

        #true_approach_date = datetime.datetime.strptime(approach_date_str, '%Y-%m-%d %H:%M:%S')#remove time xx:xx






        #neo_data = close_approach_data[0]
        #close_approach_date = neo_data.get('close_approach_date', "")
        #miss_distance = neo_data['miss_distance']['kilometers']
        return cls(orbiting_body, approach_date, miss_distance, relative_velocity, neo_id)


neo_id_to_fetch = "3542511"
neo_id = "3542511"
close_approach_data = {'close_approach_date': '1949-07-04',
                 'close_approach_date_full': '1949-Jul-04 22:19',
                 'epoch_date_close_approach': -646710060000,
                 'miss_distance': {'astronomical': '0.419726307',
                                   'kilometers': '62790161.51016609',
                                   'lunar': '163.273533423',
                                   'miles': '39015997.166588442'},
                 'orbiting_body': 'Earth',
                 'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                       'kilometers_per_second': '20.4193538659',
                                       'miles_per_hour': '45676.0526626122'}} # first entery dict from site

r = CloseApproach.close_approach_data(close_approach_data, "3542511")# yaasssss

#print(asteroid.close_approach_date)
#print(asteroid.miss_distance)
#print(asteroid.orbiting_body)
#print(asteroid.relative_velocity)
#print(asteroid.id)


## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [ ]:
# Ex. 4 your code here

class CloseApproach():
    def __init__(self, orbiting_body, approach_date, miss_distance, relative_velocity, neo_id):
        self.neo_id = neo_id
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity

    @classmethod
    def close_approach_data(cls, close_approach_data, neo_id):
        orbiting_body = close_approach_data['orbiting_body'] #if error do not change
        approach_date = close_approach_data['close_approach_date']
        miss_distance = float(close_approach_data['miss_distance']['kilometers'])
        relative_velocity = float(close_approach_data['relative_velocity']['kilometers_per_second'])
        neo_id = neo_id

        return cls(orbiting_body, approach_date, miss_distance, relative_velocity, neo_id)

class Asteroid():
    def __init__(self, close_approach_date, miss_distance, orbiting_body, relative_velocity, id):
        self.close_approach_date = close_approach_date
        self.miss_distance = miss_distance
        self.orbiting_body = orbiting_body
        self.relative_velocity = relative_velocity
        self.close_approaches = []  # Initialize the list of close_approaches

    @classmethod
    def from_NEO(cls, neo_id):
        API_KEY = 'DEMO_KEY'

        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
        print(url)#URL error check
        response = requests.get(url)
        print(response)#responce check
        neo_data = json.loads(response.text)


        close_approaches_data = neo_data.get('close_approach_data', [])

        asteroid = cls(
            neo_data['close_approach_data'][0]['close_approach_date'], #if error do not change
            neo_data['close_approach_data'][0]['miss_distance']['kilometers'],
            neo_data['close_approach_data'][0]['orbiting_body'],
            neo_data['close_approach_data'][0]['relative_velocity']['kilometers_per_second'],
            neo_id
        )

        asteroid.close_approaches = [
            CloseApproach.close_approach_data(approach_data, neo_id)
            for approach_data in close_approaches_data
        ]

        return asteroid

    def nearest_miss(self):
        if not self.close_approaches:
            return None

        closest_approach = min(self.close_approaches, key=lambda approach: approach.miss_distance)

        return closest_approach#WIP


asteroid_inst = Asteroid.from_NEO('3542517')
asteroid_inst = Asteroid.from_NEO('3542519')
asteroid_inst = Asteroid.from_NEO('3542520')#3542519,
nearest_miss_approach = asteroid_inst.nearest_miss()


print("Nearest Miss Approach:")
print(f"Orbiting Body: {nearest_miss_approach.orbiting_body}")#Name is not a member of None error
print(f"Approach Date: {nearest_miss_approach.approach_date}") #if error do not change
print(f"Miss Distance: {nearest_miss_approach.miss_distance} km")
print(f"Relative Velocity: {nearest_miss_approach.relative_velocity} km/s")
print(f"id: {nearest_miss_approach.neo_id}" )
print("of all 3")





## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here

class CloseApproach():
    def __init__(self, orbiting_body, approach_date, miss_distance, relative_velocity, neo_id):
        self.neo_id = neo_id
        self.orbiting_body = orbiting_body
        self.approach_date = approach_date
        self.miss_distance = miss_distance
        self.relative_velocity = relative_velocity

    @classmethod
    def close_approach_data(cls, close_approach_data, neo_id):
        orbiting_body = close_approach_data['orbiting_body'] #if error do not change
        approach_date = close_approach_data['close_approach_date']
        miss_distance = float(close_approach_data['miss_distance']['kilometers'])
        relative_velocity = float(close_approach_data['relative_velocity']['kilometers_per_second'])
        neo_id = neo_id

        return cls(orbiting_body, approach_date, miss_distance, relative_velocity, neo_id)

    def impact_force(self, impact_speed=20.0):
        return self.neo.estimated_mass() * impact_speed * gravitational_acceleration

class Asteroid():
    def __init__(self, close_approach_date, miss_distance, orbiting_body, relative_velocity, id):
        self.close_approach_date = close_approach_date
        self.miss_distance = miss_distance
        self.orbiting_body = orbiting_body
        self.relative_velocity = relative_velocity
        self.close_approaches = []  # Initialize the list of close_approaches

    @classmethod
    def from_NEO(cls, neo_id):
        API_KEY = 'DEMO_KEY'

        url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
        print(url)#URL error check
        response = requests.get(url)
        print(response)#responce check
        neo_data = json.loads(response.text)


        close_approaches_data = neo_data.get('close_approach_data', [])

        asteroid = cls(
            neo_data['close_approach_data'][0]['close_approach_date'], #if error do not change
            neo_data['close_approach_data'][0]['miss_distance']['kilometers'],
            neo_data['close_approach_data'][0]['orbiting_body'],
            neo_data['close_approach_data'][0]['relative_velocity']['kilometers_per_second'],
            neo_id
        )

        asteroid.close_approaches = [
            CloseApproach.close_approach_data(approach_data, neo_id)
            for approach_data in close_approaches_data
        ]

        return asteroid

    def nearest_miss(self):
        return min(self.close_approaches, key=lambda approach: approach.miss_distance)#class

    def estimated_mass(self):#

        density_of_asteroid = 2.0
        radius = self.diameter / 2.0
        diameter_cm = self.diameter * 1000.0

        volume = (4 / 3) * 3.1415926535 * (radius ** 3)

        mass = volume * density_of_asteroid

        return mass

    def closest_earth_approach(self):
        if not self.close_approaches:
            return None

        closest_approach = min(self.close_approaches, key=lambda approach: approach.miss_distance)#class
        return closest_approach



#new_
class most_dangerous_approach():
    def most_dangerous_approach(start_date, end_date):
        asteroids = [
            Asteroid.from_NEO(neo_id=neo_id, diamete=diameter)#diamete=diameter
            for neo_id, diameter in zip(neo_ids, diameters)
        ]

        hazardous_asteroids = [asteroid for asteroid in asteroids if asteroid.is_potentially_hazardous()]
        most_dangerous_approach = min(hazardous_asteroids, key=lambda asteroid: asteroid.closest_earth_approach().miss_distance)#class
        return most_dangerous_approach

